<a href="https://colab.research.google.com/github/mon-project-repos/ReviewGenerator/blob/master/ReviewGenerator_gensimWordVec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# set up session 
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)


import os
import tensorflow as tf
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ERROR: Not connected to a TPU runtime


In [0]:
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
import re


import matplotlib.image  as mpimg
#import matplotlib.pyplot as plt
#%matplotlib ipympl
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

import nltk
from nltk.stem.snowball import SnowballStemmer
# nltk.download('stopwords')
#nltk.download('punkt')
import gensim

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
percent_validation = 0.2
max_vocab_words = 20000
len_ngram = 20

#Load Data and Construct User and Product Docs

In [0]:
dta = pickle.load(open("gdrive/My Drive/dta_review.pkl",'rb'))
dta = dta.loc[dta['prod_total']>100]




prod_unique = dta.product_id.unique()
newprodid = pd.concat( [pd.Series(prod_unique) , pd.Series(range( len(prod_unique)) )], axis=1)
newprodid.columns = ['product_id','new_product_id']
dta = pd.merge(dta, newprodid, how='left', on='product_id')


class_unique = dta.text_class_id.unique()
newclassid = pd.concat([pd.Series(class_unique), pd.Series(range(len(class_unique)))], axis=1)
newclassid.columns = ['text_class_id', 'new_class_id']
dta = pd.merge(dta, newclassid, how='left', on='text_class_id')


reviewer_unique = dta.reviewer_id.unique()
newreviewerid = pd.concat([pd.Series(reviewer_unique),
                           pd.Series(range(len(reviewer_unique)))], axis=1)
newreviewerid.columns = ['reviewer_id', 'new_reviewer_id']
dta = pd.merge(dta, newreviewerid, how='left', on='reviewer_id')


review_unique = dta.review_id.unique()
newreviewid = pd.concat([pd.Series(review_unique),
                           pd.Series(range(len(review_unique)))], axis=1)
newreviewid.columns = ['review_id', 'new_review_id']
dta = pd.merge(dta, newreviewid, how='left', on='review_id')

#dta = dta.drop(columns=['id_x','id_y'])
#print(dta.head())
#print(dta.shape)
#print(dta.loc[dta['new_reviewer_id'==15],:].head())

n_users, n_products = len(dta.new_reviewer_id.unique()),  len(dta.new_product_id.unique())





# Pre process NLP

In [0]:


min_word_occur = 1
word_vector_dim = 100
stemWords = False
stemmer = SnowballStemmer('english')
#from gensim.models import Word2Vec
if stemWords:
    word_list = [[ stemmer.stem(wd) 
              for wd in tf.keras.preprocessing.text.text_to_word_sequence(x)]
             for x in dta['review_data'] ]
else:
     word_list = [ tf.keras.preprocessing.text.text_to_word_sequence(x)
             for x in dta['review_data'] ]
        
# gensim word2vec can't handle oov
if False:
    w2v = gensim.models.Word2Vec(word_list,
                            iter=10,
                            min_count = min_word_occur,
                            size= word_vector_dim,
                            workers = 4)
else:
    w2v = gensim.models.FastText(word_list, 
                                 iter=10,
                                 min_count=min_word_occur, 
                                 size=word_vector_dim,
                                workers=4)
#w2v.wv.vocab


In [86]:
bkvec=w2v['shoes']
print(w2v.wv.most_similar([bkvec]))
# shoe and shoes are different.  stemming could help (Fast text seems to fix this)
print(bkvec)
print(w2v.wv.most_similar([w2v.wv['scram']]))
print(len(w2v.wv.vocab))

[('shoes', 0.9999999403953552), ('sneakers', 0.9428362846374512), ('peltzshoes', 0.9379158616065979), ('shoes…i', 0.9259605407714844), ('shoes❤', 0.9254558086395264), ('shoes😻', 0.9252068400382996), ('shoes😀', 0.9249280691146851), ('shoes😎', 0.9247919321060181), ('shoes…', 0.9245895147323608), ('shoes\u200b', 0.9245272874832153)]
[ 1.6325024e+00 -6.0978085e-01  6.2828052e-01  1.9065645e+00
  9.1431588e-02  2.1414421e+00  1.9143313e+00 -7.5646979e-01
 -1.7813648e-01  6.5749681e-01  1.1391232e+00  5.3198487e-01
  8.3169065e-02  1.6458479e+00  1.5751919e+00  3.2982688e+00
 -1.7501521e+00  1.5673068e+00 -4.8112124e-01 -2.6894319e+00
  9.6259475e-01  1.2730095e+00 -1.3554987e+00  5.9053612e-01
 -8.9043462e-01 -1.5819083e+00 -6.6028142e-01 -5.3892112e-01
 -6.4651012e-01  6.3784391e-01  1.7346882e+00  1.0030652e+00
  8.7816131e-01  1.6560673e+00 -4.5222959e-01 -4.2138296e-01
  9.5193706e-02  6.0294050e-01  9.7305059e-02 -1.9090381e+00
 -2.0317075e+00  2.8470609e+00  9.0418762e-01 -1.5644860e+

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [143]:
stem_wd = 'have'
print(w2v.wv.vocab)
print(dir(w2v.wv.vocab))
#print('ry get', w2v.wv.vocab.__getitem__(16))
print(w2v.wv.vocab[stem_wd])
print(w2v.wv.vocab[stem_wd].__class__)
print(dir(w2v.wv.vocab[stem_wd]))
print('index', w2v.wv.vocab[stem_wd].index)
print(w2v.wv.index2word[16])
#w2v.wv.vocab[word_list[1][0]].index

#review = word_list[0]
#[w2v.wv.vocab[stem_wd] for stem_wd in review]

{'i': <gensim.models.keyedvectors.Vocab object at 0x7f4574737278>, 'have': <gensim.models.keyedvectors.Vocab object at 0x7f45747372e8>, 'tried': <gensim.models.keyedvectors.Vocab object at 0x7f4574737358>, 'all': <gensim.models.keyedvectors.Vocab object at 0x7f4574737400>, 'different': <gensim.models.keyedvectors.Vocab object at 0x7f4574737438>, 'styles': <gensim.models.keyedvectors.Vocab object at 0x7f4574737470>, 'of': <gensim.models.keyedvectors.Vocab object at 0x7f45747374a8>, 'asics': <gensim.models.keyedvectors.Vocab object at 0x7f4574737518>, 'for': <gensim.models.keyedvectors.Vocab object at 0x7f4574737588>, 'running': <gensim.models.keyedvectors.Vocab object at 0x7f4574737630>, 'shoes': <gensim.models.keyedvectors.Vocab object at 0x7f4574737668>, 'contend': <gensim.models.keyedvectors.Vocab object at 0x7f45747376d8>, 'cumulus': <gensim.models.keyedvectors.Vocab object at 0x7f4574737710>, 'nimbus': <gensim.models.keyedvectors.Vocab object at 0x7f4574737780>, 'scram': <gensim.mo

In [0]:

# append sequences (integer lists) to dta so we can use a groupby
sequences = [[w2v.wv.vocab[stem_wd].index 
              for stem_wd in review] 
             for review in word_list]
dta['sequences'] = sequences
# #-----------

# first_validate = int(len(dta)*(1-percent_validation))
# print('Total',dta.shape, 'first validate:' ,first_validate)

# review_train = dta['review_data'].iloc[:first_validate]
# review_validate = dta['review_data'].iloc[first_validate: ]
# #corpus_train = (" end-of-review ").join( )
# #corpus_validation = (" end-of-review ").join( dta['review_data'].iloc[first_validate:])
# dta = None

In [0]:

# tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_vocab_words+1,
#                                                   lower=True, 
#                                                   filters="",
#                                                  oov_token="<OOV>",
#                                                   char_level=False)

In [0]:



# corpus_train = (" endOfReview/n").join(review_train)
# tokenizer.fit_on_texts(tf.keras.preprocessing.text.text_to_word_sequence(corpus_train))
# total_words = len(tokenizer.word_index) + 1

# word_freq = tokenizer.word_counts
# word_idx = tokenizer.word_index
# print(word_idx)
# print(word_freq)

# #print(len(tokenizer.word_index))
# print(len(word_freq))

# idx_word = {v:k for k,v in word_idx.items()}
# print(idx_word)
# print(len(idx_word))

In [113]:
#-----------------



# group documents by userId or productId, set to max num words and pad sequence
def joinAndPad(x, max_len ):
  doc = sum (list(x['sequences']) ,[])
  padded_doc = pad_sequences([doc], maxlen=max_len)
  return padded_doc


prod_lens = dta.groupby('new_product_id').apply( lambda x:  len( sum(x['sequences'],[]) ))

user_lens = dta.groupby('new_reviewer_id').apply( lambda x:  len( sum(x['sequences'],[]) ))
print('prodDocs', 'max: ', max(prod_lens), 'min: ' ,min(prod_lens), 'avg:', np.mean(prod_lens), np.quantile(prod_lens,.75), np.quantile(prod_lens,.95))
print('userDocs', 'max: ', max(user_lens), 'min: ',min(user_lens),  'avg:', np.mean(user_lens), np.quantile(user_lens,.75), np.quantile(user_lens,.95))

max_len_prod = 5000
max_len_user = 25

docByProd = dta.groupby('new_product_id').apply(lambda x: joinAndPad(x ,max_len=max_len_prod) )
docByUser = dta.groupby('new_reviewer_id').apply(lambda x: joinAndPad(x, max_len=max_len_user))
wordByUser = dta.groupby('new_reviewer_id').apply(lambda x: ("<eor>").join(x['review_data']))
#dta3.head()
print('prodDocs', len(docByProd))
print('userDocs', len(docByUser))



# #dta3.head()
# print('prodDocs', len(docByProd))
# print('userDocs', len(docByUser))

prodDocs max:  156563 min:  1893 avg: 24944.44318181818 31448.5 119672.59999999995
userDocs max:  984 min:  0 avg: 28.69613700241846 36.0 93.0
prodDocs 88
userDocs 76495


In [146]:
ielem = 0
print('first doc', docByUser[ielem])

#print(dta.loc[dta['new_reviewer_id']==0]['sequences'][0])
print('reveresed sentence', " ".join([ w2v.wv.index2word[i] for i in docByUser[ielem][0]]))
#print( tokenizer.texts_to_sequences([review_train[0]])[0])
print(wordByUser[ielem])
print(dta.iloc[0]['review_data'])
print(dta.iloc[0]['sequences'])
print(" ".join([ w2v.wv.index2word[i] for i in dta.iloc[0]['sequences'] ]))

first doc [[   7   38 2845    8   32  217   86    0   50  358  297    2    5   11
    22  101  157    1  720  297   16   70  427   48   41]]
reveresed sentence these were bothering my feet while running i bought gel inserts and they are so much better the original inserts have no cushioning at all
I have tried all different styles of Asics for running shoes, Contend, Cumulus, Nimbus, Scram,  GT-1000, and GT-2000. All have been great shoes with the exception of the Scram which are technically trail runners but they were just to narrow in the toe box. These are a bit stiffer than other Asics but I hope that means they will hold up. They don't have as much cushion as the Cumulus or Nimbus and I will be going back to one of those for running. These are still good shoes, I just prefer the others.Edit:  These were bothering my feet while running. I bought gel inserts and they are so much better! The original inserts have no cushioning at all. 
I have tried all different styles of Asics for r

In [0]:
def get_sequences(reviews,  len_ngram=20):
    sequences = []
    review_sentences = " ".join(reviews)
    review_tokens = tokenizer.texts_to_sequences([review_sentences])[0] #map each word to word_index 
    #for review in review_sentences:
        #review_words = review + " endOfReview/n"
        #tokenized_review = tokenizer.texts_to_sequences([review_words])[0] #map each word to word_index 
        #print(len(tokenized_review))
    #for i in range(1, min( [maxlen_ngram,len(review_words)])): # make ngrams
    for i in range(len_ngram, len(review_tokens)):
        sequences.append( review_tokens[i-len_ngram:i] )
    #if max_review_len is None:
    #    max_review_len = max([len(x) for x in sequences])
    #print(max_review_len)
    #sequences = np.array(pad_sequences(sequences,
    #                                     maxlen=maxlen_ngram,
    #                                     padding='pre')) 
    return np.array(sequences) #, max_review_len
        
def get_pred_lastword(sequences):
    predictors, last_word = sequences[:,:-1] , sequences[:,-1]
    #last_word = tf.keras.utils.to_categorical(last_word, num_classes=max_vocab_words+1)
    return (predictors, last_word )

        


In [0]:
train_sequences = get_sequences(review_train,len_ngram=len_ngram)
pred_train, lastword_train = get_pred_lastword(train_sequences)

validate_sequences = get_sequences(review_validate, len_ngram=len_ngram)
pred_validate, lastword_validate = get_pred_lastword(validate_sequences)

In [0]:
print('x=', pred_train.shape, 'y=', lastword_train.shape)
print(pred_train[:3])
print(lastword_train[:3])
print(pred_train.shape)
print(lastword_train.shape)
for m in range(24):
    print((" ").join([idx_word[i] if i>0 else "" for i in train_sequences[m]]))

print(review_train[0])